# Data Exploration in Cloud 

In [95]:
import os
import boto3
import pickle
import requests
import pandas as pd
import numpy as np
import shutil


In [14]:
# Let's use Amazon S3
s3 = boto3.resource('s3')
for bucket in s3.buckets.all():
    print(bucket.name)

pablo-mushroom-bucket


## Downloading Dataframe from S3
Here we get the cleaned dataframe with all the metadata about all the pictures. From it we can create the file directory tree in S3, and download all the appropriate images to it.

In [15]:
s3.Bucket('pablo-mushroom-bucket').download_file(
                            'combined_dataset_df.p', 'data.p')
df = pickle.load(open('data.p','rb'), encoding='latin1')

#creating truly unique identifier for each image
df['Unique_ID'] = \
    df['Data_Source'].apply(lambda x: x + '_') \
    + df['id'].apply(lambda x: str(x)) 

We will define some useful functions for doing this. Essentially creating the appropriate file directory, generate the url, and transfer the appropriate image to S3. We may have to do this on a picture by picture basis for MO dataset. For DSA dataset, we will have to transfer the entire dataset which is currently stored locally on this EC2 instance. 

In [76]:
taxa_fields = ['Phylum','Class','Order','Family','Genus','Species']

def create_path_name(series):
    folders = [series.get(w).values[0] for w in taxa_fields]
    if not sum([not isinstance(w,str) for w in folders ]):
        return '/'.join(folders)
    else:
        return None
    
def generate_url_image_API_MO(image_id):
    if not np.isnan(image_id):
        url = 'https://images.mushroomobserver.org/960/' \
                                    + str(int(image_id)) + '.jpg'
        return url
    else:
        return None
    
def upload_image_S3_MO(cid, target):
    url = generate_url_image_API_MO(cid)
    r = requests.get(url, allow_redirects=True)
    open('pics/test.jpg', 'wb').write(r.content)

    # Upload a new file
    data = open('pics/test.jpg', 'rb')
    s3.Bucket('pablo-mushroom-bucket').put_object(Key=target, Body=data)
    
def upload_image_S3_DSA(Image, target):
    data = open(Image, 'rb')
    s3.Bucket('pablo-mushroom-bucket').put_object(Key=target, Body=data)
    
def create_target(Unique_ID):
    tmp = df[df.Unique_ID == Unique_ID]
    if create_path_name(tmp) != None:
        if tmp.shape[0] == 1:
            return 'pics/' + create_path_name(tmp) + '/' + Unique_ID + '.JPG'
        else:
            return ['pics/' + create_path_name(tmp) + '/' + Unique_ID + \
                       '_' + str(w+1) + '.JPG' for w in range(tmp.shape[0])]

def process_entry(Unique_ID):
    tmp = df[df.Unique_ID == Unique_ID]
    #for now we are being strict about picture having all fields
    if create_path_name(tmp) != None:
        if Unique_ID.startswith('D'):
            Image = df[df.Unique_ID == Unique_ID].Image[0]
            target = create_target(Unique_ID)
            upload_image_S3_DSA(Image, target)
        elif Unique_ID.startswith('M'):
            CIDs = tmp.Image.values
            targets = create_target(Unique_ID)
            if tmp.shape[0] > 1:
                for cid, target in zip(CIDs, targets):
                    upload_image_S3_MO(cid, target)
            else:
                upload_image_S3_MO(CIDs[0], targets)
        else:
            print('something went wrong...%s' %Unique_ID)
    else:
        return
    

In [8]:
display(df)

,id,location_id,in_location,vote_cache,when,Domain,Kingdom,Phylum,Class,Order,Family,Genus,Species,Image,Data_Source,Unique_ID
0,43829,-1,-1,3.000000,,Eukarya,Fungi,Basidiomycota,Agaricomycetes,Agaricales,Agaricaceae,Leucoagaricus,leucothites,images/16390_Leucoagaricus_leucothites/OBL2010...,DSA,DSA_43829
0,44039,-1,-1,3.000000,,Eukarya,Fungi,Basidiomycota,Agaricomycetes,Agaricales,Agaricaceae,Leucoagaricus,leucothites,images/16390_Leucoagaricus_leucothites/OBL2010...,DSA,DSA_44039
0,44255,-1,-1,3.000000,,Eukarya,Fungi,Basidiomycota,Agaricomycetes,Agaricales,Agaricaceae,Leucoagaricus,leucothites,images/16390_Leucoagaricus_leucothites/OBL2010...,DSA,DSA_44255
0,44614,-1,-1,3.000000,,Eukarya,Fungi,Basidiomycota,Agaricomycetes,Agaricales,Agaricaceae,Leucoagaricus,leucothites,images/16390_Leucoagaricus_leucothites/FDE2010...,DSA,DSA_44614
0,45510,-1,-1,3.000000,,Eukarya,Fungi,Basidiomycota,Agaricomycetes,Agaricales,Agaricaceae,Leucoagaricus,leucothites,images/16390_Leucoagaricus_leucothites/FDE2010...,DSA,DSA_45510
0,45649,-1,-1,3.000000,,Eukarya,Fungi,Basidiomycota,Agaricomycetes,Agaricales,Agaricaceae,Leucoagaricus,leucothites,images/16390_Leucoagaricus_leucothites/FDE2010...,DSA,DSA_45649
0,53554,-1,-1,3.000000,,Eukarya,Fungi,Basidiomycota,Agaricomycetes,Agaricales,Agaricaceae,Leucoagaricus,leucothites,images/16390_Leucoagaricus_leucothites/FDE2017...,DSA,DSA_53554
0,53745,-1,-1,3.000000,,Eukarya,Fungi,Basidiomycota,Agaricomycetes,Agaricales,Agaricaceae,Leucoagaricus,leucothites,images/16390_Leucoagaricus_leucothites/FDE2017...,DSA,DSA_53745
0,44714,-1,-1,3.000000,,Eukarya,Fungi,Basidiomycota,Agaricomycetes,Agaricales,Agaricaceae,Leucoagaricus,leucothites,images/16390_Leucoagaricus_leucothites/BWP2010...,DSA,DSA_44714
0,44851,-1,-1,3.000000,,Eukarya,Fungi,Basidiomycota,Agaricomycetes,Agaricales,Agaricaceae,Leucoagaricus,leucothites,images/16390_Leucoagaricus_leucothites/BWP2010...,DSA,DSA_44851


## Let the Transfer Begin
I tested each type of file individually. Now let's transfer/download our dataset. 

In [ ]:
J = 7338
for i, Unique_ID in enumerate(df.Unique_ID.iloc[J:]):
    if i%5000 == 0:
        print('Working on entry %i/%i' %(i, df.Unique_ID.shape[0]))
    process_entry(Unique_ID)


Working on entry 0/716900


UPDATE: So it takes ~250ms for each process_entry(Unique_ID) call, which means the entire dataset will go for about 45 hours. This could be trouble/ I really don't think this is the proper way to use the cloud. Also, this notebook keeps disconnecting and messing up the transfer. I switched to using tmux and a script, so at least it won't get interrupted. But really, we're going to have to do this locally. I hope I don't run out of memory. 

In [75]:

    
def MO_write_image(cid, target):
    url = generate_url_image_API_MO(cid)
    r = requests.get(url, allow_redirects=True)
    open(target, 'wb').write(r.content)
    return
    
def DSA_transfer_image(Image, target):
    copyfile(Image, target)
    return

def create_target(Unique_ID):
    tmp = df[df.Unique_ID == Unique_ID]
    if create_path_name(tmp) != None:
        if tmp.shape[0] == 1:
            return 'pics/' + create_path_name(tmp) + '/' + Unique_ID + '.JPG'
        else:
            return ['pics/' + create_path_name(tmp) + '/' + Unique_ID + \
                       '_' + str(w+1) + '.JPG' for w in range(tmp.shape[0])]
    else:
        return None
        
        
def process_entry_locally(Unique_ID):
    tmp = df[df.Unique_ID == Unique_ID]
    #for now we are being strict about picture having all fields
    if create_path_name(tmp) != None:
        if Unique_ID.startswith('D'):
            Image = df[df.Unique_ID == Unique_ID].Image[0]
            target = create_target(Unique_ID)
            print(Image)
            print(target)
            DSA_transfer_image(Image, target)
        elif Unique_ID.startswith('M'):
            CIDs = tmp.Image.values
            targets = create_target(Unique_ID)
            if tmp.shape[0] > 1:
                for cid, target in zip(CIDs, targets):
                    MO_write_image(cid, target)
            else:
                MO_write_image(CIDs[0], targets)
        else:
            print('something went wrong...%s' %Unique_ID)
    else:
        return
    

In [22]:
Unique_ID = df.Unique_ID.iloc[20000]
print(Unique_ID)
print(df[df.Unique_ID == Unique_ID])
process_entry_locally(Unique_ID)
# Unique_ID = 'MO_355353'
# print(Unique_ID)
# %timeit process_entry_locally(Unique_ID)

DSA_8217
     id  location_id  in_location  vote_cache when   Domain Kingdom  \
0  8217           -1           -1         3.0       Eukarya   Fungi   

          Phylum           Class       Order Family      Genus Species  \
0  Basidiomycota  Agaricomycetes  Agaricales    NaN  Clitocybe   odora   

                                               Image Data_Source Unique_ID  
0  images/11793_Clitocybe_odora var. odora /TKH20...         DSA  DSA_8217  


In [14]:
(439592 * 0.05) / (60 * 60)

6.105444444444445

In [7]:
print(np.random.random(1))

[0.13641263]


# MVP
For a minimal viable product, we will use tensorflow's example of retraining an Inception V3 (https://www.tensorflow.org/hub/tutorials/image_retraining) on our categories. I think it might be easier to start with DSA dataset that is already flat, and feed it as much of MO's data as we can. We will then only include categories that have at least 100 pictures, re-train, and use model to load into website. 

In [36]:
subfolders = os.listdir(os.getcwd() + '/images')
species_to_dir = {w : w.split('_')[1:] for w in subfolders}
idx = pd.IndexSlice

In [39]:
%timeit (df[['Genus','Species']].iloc[0])
%timeit ['Disciotis', 'venosa'] in species_to_dir.values()

9.96 ms ± 23.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
254 ns ± 0.0757 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [53]:
MO_only = df.loc[df.Data_Source=='MO',['Genus','Species']]
def f(x,y):
    if isinstance(x,str) and isinstance(y,str):
        return [x,y] in species_to_dir.values()
    else:
        return False
    
filter_MO_in_DSA = MO_only.apply(lambda x: f(*x), axis=1)


In [60]:
MO_only = df.loc[df.Data_Source=='MO',:]
MO_only = MO_only[filter_MO_in_DSA]
MO_only['GS_list'] = [[a,b] for a,b in zip(MO_only['Genus'],MO_only['Species'])]
display(MO_only)

,id,location_id,in_location,vote_cache,when,Domain,Kingdom,Phylum,Class,Order,Family,Genus,Species,Image,Data_Source,Unique_ID,GS_list
0,1,214,1,1.92335,['2004-07-13'],Eukarya,Fungi,Ascomycota,Sordariomycetes,Xylariales,Xylariaceae,Xylaria,polymorpha,1,MO,MO_1,"[Xylaria, polymorpha]"
0,3,60,1,2.49991,['2002-01-08'],Eukarya,Fungi,Ascomycota,Sordariomycetes,Xylariales,Xylariaceae,Xylaria,hypoxylon,3,MO,MO_3,"[Xylaria, hypoxylon]"
0,4,5,1,2.49991,['1996-01-15'],Eukarya,Fungi,Ascomycota,Sordariomycetes,Xylariales,Xylariaceae,Xylaria,hypoxylon,4,MO,MO_4,"[Xylaria, hypoxylon]"
0,6,58,1,2.50528,['2002-01-08'],Eukarya,Fungi,Basidiomycota,Agaricomycetes,Agaricales,Typhulaceae,Xeromphalina,campanella,6,MO,MO_6,"[Xeromphalina, campanella]"
0,13,58,0,2.75106,['2005-01-07'],Eukarya,Fungi,Basidiomycota,Agaricomycetes,Agaricales,Pluteaceae,Volvopluteus,gloiocephalus,16,MO,MO_13,"[Volvopluteus, gloiocephalus]"
0,16,5,1,1.89725,['1996-01-15'],NaN,Protozoa,Myxomycota,Myxomycetes,Physarales,Physaraceae,Fuligo,septica,19,MO,MO_16,"[Fuligo, septica]"
0,20,39,1,2.73580,['2004-11-26'],Eukarya,Fungi,Basidiomycota,Agaricomycetes,Agaricales,Mycenaceae,Panellus,stipticus,26,MO,MO_20,"[Panellus, stipticus]"
0,32,58,0,2.36365,['2005-01-07'],Eukarya,Fungi,Basidiomycota,Agaricomycetes,Agaricales,Typhulaceae,Tricholomopsis,rutilans,39,MO,MO_32,"[Tricholomopsis, rutilans]"
0,35,58,0,NaN,['2005-01-07'],Eukarya,Fungi,Basidiomycota,Agaricomycetes,Agaricales,Tricholomataceae,Tricholoma,saponaceum,42,MO,MO_35,"[Tricholoma, saponaceum]"
0,42,58,0,1.67790,['2005-01-07'],Eukarya,Fungi,Basidiomycota,Agaricomycetes,Agaricales,Tricholomataceae,Tricholoma,equestre,49,MO,MO_42,"[Tricholoma, equestre]"


In [123]:

def get_folder_species(GS_list):
    species_index = list(species_to_dir.values()).index(GS_list)
    return list(species_to_dir.keys())[species_index]

def MO_write_image(url, target):
    r = requests.get(url, allow_redirects=True)
    open(target, 'wb').write(r.content)
    return


def process_entry(i):
    Genus = MO_only.iloc[i].Genus
    Species = MO_only.iloc[i].Species
    image_id = MO_only.iloc[i].Image
    if isinstance(Genus, str) and isinstance(Species, str):
        print('_'.join([Genus, Species]))
        target_dir = os.environ['HOME'] + '/images/' \
                        + '_'.join([Genus, Species]) \
                        + '/' 
        file_name = MO_only.iloc[i].Unique_ID + '.JPG'
        if not os.path.isdir(target_dir):
            os.mkdir(target_dir)

        target = target_dir + file_name

        if not os.path.isfile(target):
            #url = generate_url_image_API_MO(image_id)
            #MO_write_image(url, target)
            pass
    return



In [80]:
print(MO_only.shape)
print((0.2 * MO_only.shape[0]) / (60 * 60))
print((0.1 * MO_only.shape[0]))


(149103, 17)
8.2835
14910.300000000001


In [106]:
J = 41201
for i in range(J, MO_only.shape[0]):
    if i%1000 == 0:
        print('downloading picture %i/%i' %(i,MO_only.shape[0]))
    process_entry(i)
    J = i

downloading picture 43000/149103
downloading picture 44000/149103
downloading picture 45000/149103
downloading picture 46000/149103
downloading picture 47000/149103
downloading picture 48000/149103
downloading picture 49000/149103
downloading picture 50000/149103
downloading picture 51000/149103
downloading picture 52000/149103
downloading picture 53000/149103
downloading picture 54000/149103
downloading picture 55000/149103
downloading picture 56000/149103
downloading picture 57000/149103
downloading picture 58000/149103
downloading picture 59000/149103
downloading picture 60000/149103
downloading picture 61000/149103
downloading picture 62000/149103
downloading picture 63000/149103
downloading picture 64000/149103
downloading picture 65000/149103
downloading picture 66000/149103
downloading picture 67000/149103
downloading picture 68000/149103
downloading picture 69000/149103
downloading picture 70000/149103
downloading picture 71000/149103
downloading picture 72000/149103
downloadin

In [98]:
subfolders = os.listdir('aside_for_now/')
not_enough = {}
for subfolder in subfolders:
    species_folder = os.listdir('images/' + subfolder)
    if len(species_folder) < 100:
        not_enough[subfolder] = len(subfolder)

In [97]:
for folder in not_enough.keys():
    shutil.move('images/' + folder, 'aside_for_now/' + folder)

In [105]:
subfolders = os.listdir('aside_for_now/')
for folder in subfolders:
    shutil.move('aside_for_now/' + folder, 'images/' + folder)

## Download the Rest of the Data
OK. Let's finish downloading the rest of the MO dataset. This should be running continuously. Once it's finished, transfer to S3. This should be left running today. 

In [107]:
display(df)

,id,location_id,in_location,vote_cache,when,Domain,Kingdom,Phylum,Class,Order,Family,Genus,Species,Image,Data_Source,Unique_ID
0,43829,-1,-1,3.000000,,Eukarya,Fungi,Basidiomycota,Agaricomycetes,Agaricales,Agaricaceae,Leucoagaricus,leucothites,images/16390_Leucoagaricus_leucothites/OBL2010...,DSA,DSA_43829
0,44039,-1,-1,3.000000,,Eukarya,Fungi,Basidiomycota,Agaricomycetes,Agaricales,Agaricaceae,Leucoagaricus,leucothites,images/16390_Leucoagaricus_leucothites/OBL2010...,DSA,DSA_44039
0,44255,-1,-1,3.000000,,Eukarya,Fungi,Basidiomycota,Agaricomycetes,Agaricales,Agaricaceae,Leucoagaricus,leucothites,images/16390_Leucoagaricus_leucothites/OBL2010...,DSA,DSA_44255
0,44614,-1,-1,3.000000,,Eukarya,Fungi,Basidiomycota,Agaricomycetes,Agaricales,Agaricaceae,Leucoagaricus,leucothites,images/16390_Leucoagaricus_leucothites/FDE2010...,DSA,DSA_44614
0,45510,-1,-1,3.000000,,Eukarya,Fungi,Basidiomycota,Agaricomycetes,Agaricales,Agaricaceae,Leucoagaricus,leucothites,images/16390_Leucoagaricus_leucothites/FDE2010...,DSA,DSA_45510
0,45649,-1,-1,3.000000,,Eukarya,Fungi,Basidiomycota,Agaricomycetes,Agaricales,Agaricaceae,Leucoagaricus,leucothites,images/16390_Leucoagaricus_leucothites/FDE2010...,DSA,DSA_45649
0,53554,-1,-1,3.000000,,Eukarya,Fungi,Basidiomycota,Agaricomycetes,Agaricales,Agaricaceae,Leucoagaricus,leucothites,images/16390_Leucoagaricus_leucothites/FDE2017...,DSA,DSA_53554
0,53745,-1,-1,3.000000,,Eukarya,Fungi,Basidiomycota,Agaricomycetes,Agaricales,Agaricaceae,Leucoagaricus,leucothites,images/16390_Leucoagaricus_leucothites/FDE2017...,DSA,DSA_53745
0,44714,-1,-1,3.000000,,Eukarya,Fungi,Basidiomycota,Agaricomycetes,Agaricales,Agaricaceae,Leucoagaricus,leucothites,images/16390_Leucoagaricus_leucothites/BWP2010...,DSA,DSA_44714
0,44851,-1,-1,3.000000,,Eukarya,Fungi,Basidiomycota,Agaricomycetes,Agaricales,Agaricaceae,Leucoagaricus,leucothites,images/16390_Leucoagaricus_leucothites/BWP2010...,DSA,DSA_44851


In [138]:
print(df.shape)

(716900, 16)


In [137]:
subfolders = os.listdir('images')
rename_DSA_dirs = {w:'_'.join(w.split('_')[1:]) \
                    for w in subfolders if len(w.split('_')) == 3}
for old_folder, new_folder in rename_DSA_dirs.items():
    try:
        os.rename('images/' + old_folder, 'images/' + new_folder)
    except OSError:
        for efile in os.listdir('images/' + new_folder):
            shutil.copyfile('images/' + old_folder + '/' + efile,
                            'images/' + new_folder + '/' + efile)

FileNotFoundError: [Errno 2] No such file or directory: 'images/65022_Inocybe_rimosa/MMN2009PIC67091157.JPG'

In [134]:
print(os.listdir('images/Disciotis_venosa'))
os.rename('images/13361_Disciotis_venosa', 'images/Disciotis_venosa')

['TAK2014PIC66033070.JPG', 'TKH2014PIC65452468.JPG', 'FDE2014PIC29433950.JPG', 'GW2017-9190897_ryjK2AoCe.JPG', 'JHV2013PIC66245952.JPG', 'JAH2017-9190365_HkghRh5zCl.JPG', 'TAK2017-9188993_SJlddXhDhg.JPG', 'JHV2017-9189223_HJRGwO6ne.JPG', 'TKH2012PIC61132275.JPG', 'MC2017-9191253_rJftB_ACg.JPG', 'TAK2013PIC45890140.JPG', 'JHV2012PIC24507660.JPG', 'TAK2014PIC52502888.JPG', 'TAK2016PIC74520414.JPG', 'JHV2009PIC20866082.JPG', 'FDE2017-9190521_SkZLPKuLCg.JPG', 'TAK2014PIC74571914.JPG', 'TSJ2017-9190303_BkgKCB-0e.JPG', 'FDE2014PIC68719186.JPG', 'JM2012PIC35653198.JPG', 'JMI2017-9190278_BkeiHINZ0g.JPG', 'JAH2014PIC51722967.JPG', 'TS2017-9190503_H17GAA7LRx.JPG', 'TAK2017-9188993_SyZOOXnw2l.JPG', 'EBC2015PIC85794172.JPG', 'TOB2017-9191361_H1n-n9e1b.JPG', 'TAK2015PIC53493338.JPG', 'JHV2017-9189223_HkWAzPd62x.JPG', 'LASSE2017-9189885_H1HE2EYpe.JPG', 'MC2017-9191253_S1gJm7T1yb.JPG', 'TAK2014PIC15058002.JPG', 'TAK2017-9188993_ByOu7nP2l.JPG', 'AE2014PIC37730395.JPG', 'FDE2017-9190521_rJUPYO8Ce.JPG',

FileNotFoundError: [Errno 2] No such file or directory: 'images/13361_Disciotis_venosa' -> 'images/Disciotis_venosa'

In [124]:
MO_only = df.loc[df.Data_Source=='MO',:]
J = 0
for i in range(J, MO_only.shape[0]):
    if i%1000 == 0:
        print('downloading picture %i/%i' %(i,MO_only.shape[0]))
    process_entry(i)
    J = i
    

downloading picture 0/668604
Xylaria_polymorpha
Xylaria_hypoxylon
Xylaria_hypoxylon
Xeromphalina_campanella
Volvopluteus_gloiocephalus
Verpa_bohemica
Fuligo_septica
Panellus_stipticus
Hexagonia_hydnoides
Hexagonia_hydnoides
Gymnopus_bakerensis
Tylopilus_indecisus
Sutorius_eximius
Harrya_chromapes
Tricholomopsis_rutilans
Tricholomopsis_flavissima
Tricholoma_vernaticum
Tricholoma_saponaceum
Tricholoma_murrillianum
Tricholoma_murrillianum
Tricholoma_murrillianum
Tricholoma_zelleri
Tricholoma_equestre
Tricholoma_equestre
Tricholoma_equestre
Tricholoma_aurantium
Tricholoma_aurantium
Trametes_versicolor
Trametes_versicolor
Trametes_versicolor
Suillus_tomentosus
Suillus_tomentosus
Suillus_ponderosus
Suillus_ponderosus
Suillus_lakei
Suillus_lakei
Suillus_lakei
Suillus_caerulescens
Suillus_caerulescens
Suillus_brevipes
Suillus_brevipes
Stropharia_alboambigua
Leratiomyces_percevalii
Stropharia_ambigua
Strobilurus_trullisatus
Sparassis_radicata
Sparassis_radicata
Scleroderma_laeve
Scleroderma_cit

Bovista_pila
Bovista_pila
Bovista_pila
Lycoperdon_pyriforme
Lycoperdon_pyriforme
Pholiota_lenta
Rubroboletus_eastwoodiae
Rubroboletus_eastwoodiae
Rubroboletus_eastwoodiae
Hypomyces_lactifluorum
Hypomyces_lactifluorum
Hohenbuehelia_petaloides
Hohenbuehelia_petaloides
Hohenbuehelia_petaloides
Amanita_constricta
Amanita_constricta
Armillaria_sinapina
Armillaria_sinapina
Boletus_edulis
Boletus_edulis
Cryptoporus_volvatus
Fomitopsis_pinicola
Ganoderma_oregonense
Ganoderma_oregonense
Russula_brevipes
Russula_olivacea
Stereum_hirsutum
Trametes_versicolor
Trichaptum_abietinum
Inocybe_sororia
Inocybe_sororia
Fistulina_hepatica
Fistulina_hepatica
Agaricus_californicus
Amanita_phalloides
Amanita_phalloides
Leucoagaricus_leucothites
Panaeolus_papilionaceus
Phellinus_pomaceoides
Stereum_hirsutum
Leucoagaricus_leucothites
Leucoagaricus_leucothites
Leucoagaricus_leucothites
Agaricus_bisporus
Agaricus_bisporus
Agaricus_bisporus
Russula_brunneola
Verpa_conica
Amanita_ocreata
Phylloporus_arenicola
Suill

Hypholoma_fasciculare
Entoloma_ferruginans
Crepidotus_mollis
Crepidotus_mollis
Crepidotus_mollis
Agaricus_subrutilescens
Amanita_pseudobreckonii
Amanita_pseudobreckonii
Amanita_constricta
Amanita_augusta
Amanita_calyptroderma
Boletus_edulis
Boletus_edulis
Cantharellus_Cantharellus
Caulorhiza_umbonata
Chalciporus_piperatoides
Clitopilus_prunulus
Cortinarius_vanduzerensis
Dacrymyces_stillatus
Dacrymyces_stillatus
Entoloma_medianox
Entoloma_medianox
Fomitopsis_pinicola
Fomitopsis_pinicola
Gomphus_clavatus
Turbinellus_floccosus
Gymnopilus_sapineus
Hericium_erinaceus
Hypholoma_fasciculare
Inocybe_sororia
Inocybe_sororia
Laccaria_amethysteo-occidentalis
Lactarius_rubrilacteus
Lactarius_rubrilacteus
Laetiporus_conifericola
Laetiporus_conifericola
Mycena_capillaripes
Pleurotus_opuntiae
Russula_clavipes
Suillus_pungens
Suillus_pungens
Tricholoma_imbricatum
Tricholoma_saponaceum
Tricholomopsis_rutilans
Tricholomopsis_rutilans
Russula_aeruginea
Russula_aeruginea
Hydnum_repandum
Amanita_augusta
Am

Hydnum_repandum
Hydnum_oregonense
Hygrocybe_flavescens
Hygrocybe_miniata
Hygrocybe_miniata
Melanoleuca_melaleuca
Melanoleuca_melaleuca
Suillus_pungens
Trichaptum_abietinum
Trichaptum_abietinum
Tricholoma_saponaceum
Gomphus_clavatus
Xylodon_verruculosus
Xylodon_verruculosus
Xylodon_verruculosus
Clavariadelphus_occidentalis
Clavariadelphus_occidentalis
Laccaria_amethysteo-occidentalis
Trametes_versicolor
Chroogomphus_ochraceus
Chroogomphus_ochraceus
Pholiota_velaglutinosa
Coltricia_perennis
Coltricia_perennis
Suillus_pungens
Tapinella_panuoides
Tapinella_panuoides
Tapinella_panuoides
Hydnum_repandum
Inocybe_whitei
Inocybe_whitei
Inocybe_whitei
Amanita_constricta
Boletus_edulis
Amanita_muscaria
Amanita_muscaria
Leucopaxillus_cerealis
Leucopaxillus_cerealis
Leucopaxillus_cerealis
Entoloma_medianox
Entoloma_medianox
Schizophyllum_commune
Schizophyllum_commune
Schizophyllum_commune
Schizophyllum_commune
Cortinarius_collinitus
Entoloma_rhodopolium
Russula_basifurcata
Russula_basifurcata
Russu

Leptonia_Leptonia
Phaeolus_schweinitzii
Phaeolus_schweinitzii
Pholiota_velaglutinosa
Phylloporus_rhodoxanthus
Phylloporus_rhodoxanthus
Phylloporus_rhodoxanthus
Phylloporus_rhodoxanthus
Ramaria_formosa
Sowerbyella_rhenana
Xeromphalina_campanella
Coprinellus_micaceus
Coprinellus_micaceus
Coprinellus_micaceus
Phylloporus_rhodoxanthus
Phylloporus_rhodoxanthus
Phylloporus_rhodoxanthus
Phylloporus_rhodoxanthus
Amanita_ocreata
Amanita_ocreata
Lycoperdon_perlatum
Helvella_lacunosa
Helvella_lacunosa
Xeromphalina_campanella
Xeromphalina_campanella
Amanita_ocreata
Amanita_ocreata
Leratiomyces_percevalii
Leratiomyces_percevalii
Leratiomyces_percevalii
Clavulina_rugosa
Hygrocybe_miniata
Entoloma_Claudopus
Entoloma_Claudopus
Entoloma_Claudopus
Entoloma_Claudopus
Russula_ochroleuca
Russula_ochroleuca
Russula_ochroleuca
Russula_ochroleuca
Clitocybe_nuda
Clitocybe_nuda
Clitocybe_nuda
Aureoboletus_flaviporus
Aureoboletus_flaviporus
Aureoboletus_flaviporus
Aureoboletus_flaviporus
Cuphophyllus_pratensis
C

...

Suillus_ponderosus
Trametes_versicolor
Trametes_versicolor
Sarcoscypha_austriaca
Ochrolechia_subpallescens
Ochrolechia_subpallescens
Ochrolechia_subpallescens
Ochrolechia_subpallescens
Craterellus_fallax
Craterellus_fallax
Aureoboletus_russellii
Aureoboletus_russellii
Cortinarius_iodes
Cortinarius_iodes
Baorangia_bicolor
Entoloma_Claudopus
Entoloma_Claudopus
Gyroporus_castaneus
Gyroporus_castaneus
Marasmius_elegans
Marasmius_elegans
Marasmius_elegans
Marasmius_elegans
Marasmius_elegans
Marasmius_elegans
Entoloma_virescens
Entoloma_virescens
Entoloma_virescens
Entoloma_virescens
Entoloma_virescens
Entoloma_virescens
Cantharellus_Cantharellus
Cantharellus_Cantharellus
Cantharellus_Cantharellus
Amanita_phalloides
Amanita_phalloides
Amanita_phalloides
Amanita_phalloides
Earliella_scabrosa
Earliella_scabrosa
Earliella_scabrosa
Stereum_complicatum
Stereum_complicatum
Inocybe_geophylla
Inocybe_geophylla
Inocybe_geophylla
Inocybe_geophylla
Inocybe_geophylla
Inocybe_geophylla
Auricularia_angios

Suillus_pungens
Suillus_pungens
Suillus_pungens
Clitocybe_nuda
Clitocybe_nuda
Clitocybe_nuda
Panaeolus_subbalteatus
Panaeolus_subbalteatus
Pholiotina_rugosa
Pholiotina_rugosa
Pholiotina_rugosa
Deconica_coprophila
Deconica_coprophila
Deconica_coprophila
Deconica_coprophila
Deconica_coprophila
Deconica_coprophila
Panaeolus_papilionaceus
Panaeolus_papilionaceus
Panaeolus_papilionaceus
Panaeolus_papilionaceus
Panaeolus_papilionaceus
Panaeolus_papilionaceus
Panaeolus_papilionaceus
Protostropharia_semiglobata
Protostropharia_semiglobata
Protostropharia_semiglobata
Leratiomyces_percevalii
Leratiomyces_percevalii
Leratiomyces_percevalii
Leratiomyces_percevalii
Pholiotina_rugosa
Agaricus_campestris
Agaricus_campestris
Agaricus_campestris
Agaricus_campestris
Entoloma_Claudopus
Entoloma_Claudopus
Entoloma_Claudopus
Psathyrella_longipes
Psathyrella_longipes
Psathyrella_longipes
Psathyrella_longipes
Psathyrella_longipes
Psathyrella_longipes
Psathyrella_longipes
Tylopilus_rubrobrunneus
Tylopilus_rub

Boletus_gertrudiae
Boletus_gertrudiae
Tylopilus_plumbeoviolaceus
Agaricus_placomyces
Agaricus_placomyces
Agaricus_placomyces
Galerina_clavata
Galerina_clavata
Dacrymyces_chrysospermus
Dacrymyces_chrysospermus
Dacrymyces_stillatus
Amanita_brunnescens
Herpothallon_rubrocinctum
Herpothallon_rubrocinctum
Amanita_flavoconia
Ganoderma_megaloma
Sarcomyxa_serotina
Sarcomyxa_serotina
Trametopsis_cervina
Trametopsis_cervina
Trametopsis_cervina
Trametopsis_cervina
Pluteus_cervinus
Pluteus_cervinus
Pluteus_cervinus
downloading picture 134000/668604
Pluteus_cervinus
Gliophorus_flavus
Hydnum_repandum
Hydnum_repandum
Hydnum_repandum
Entoloma_Claudopus
Entoloma_Claudopus
Tapinella_panuoides
Tapinella_panuoides
Phyllotopsis_nidulans
Hygrophorus_boyeri
Porphyrellus_olivaceobrunneus
Porphyrellus_olivaceobrunneus
Amanita_pantherina
Amanita_pantherina
Amanita_pantherina
Amanita_pantherina
Pleurotus_pulmonarius
Pleurotus_pulmonarius
Pleurotus_pulmonarius
Pleurotus_pulmonarius
Lentinus_arcularius
Lentinus_ar

Hygrocybe_flavescens
Hygrocybe_flavescens
Trametes_versicolor
Trametes_versicolor
Stereum_complicatum
Stereum_complicatum
Stereum_complicatum
Trametes_gibbosa
Trametes_gibbosa
Trametes_gibbosa
Pluteus_cervinus
Pluteus_cervinus
Pluteus_cervinus
Pluteus_cervinus
Pluteus_cervinus
Pluteus_cervinus
Pluteus_cervinus
Picipes_badius
Picipes_badius
Picipes_badius
Picipes_badius
Clavaria_fragilis
Boletus_separans
Boletus_separans
Boletus_separans
Boletus_separans
Hyphodontia_nespori
Hyphodontia_nespori
Hyphodontia_nespori
Panus_conchatus
Panus_conchatus
Lecanora_strobilina
Lecanora_strobilina
Panaeolus_papilionaceus
Panaeolus_papilionaceus
Amanita_gioiosa
Amanita_gioiosa
Amanita_gioiosa
Copelandia_Copelandia
Copelandia_Copelandia
Copelandia_Copelandia
Copelandia_Copelandia
Copelandia_Copelandia
Copelandia_Copelandia
Dacrymyces_chrysospermus
Dacrymyces_chrysospermus
Umbilicaria_mammulata
Umbilicaria_mammulata
Umbilicaria_mammulata
Umbilicaria_mammulata
Umbilicaria_mammulata
Umbilicaria_mammulata


KeyboardInterrupt: 